In [94]:
# Note to import from .py files, must follow structure
# from <.py filename excluding '.py'> import <class name>
# Optionslam creds: aspringfastlaner Options2018

# Importing necessary models
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
import seaborn as sns
import matplotlib.dates as dates
# import matplotlib.ticker as ticker
from lxml import html
import requests
import webbrowser
from bs4 import BeautifulSoup as bs
import json
import csv
import sched, time
import pandas_datareader as datareader
from pandas_datareader.data import Options
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
%matplotlib inline

# Alpha Vantage API Key
# 5HZEUI5AFJB06BUK

# ts = TimeSeries(key='5HZEUI5AFJB06BUK', output_format='pandas')
# data, meta_data = ts.get_intraday(symbol='MSFT',interval='1min', outputsize='full')
# data['close'].plot()
# plt.title('Intraday Times Series for the MSFT stock (1 min)')
# For intraday
# https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=MSFT&interval=1min&apikey=d5HZEUI5AFJB06BUK&datatype=csv

# For daily
# https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&apikey=5HZEUI5AFJB06BUK&datatype=csv

In [255]:
'''
Pulling S&P 500 Names
'''

def pull_sp500_list():
    site = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    res = requests.get(site)
    soup = bs(res.text, 'lxml')
    table = soup.find_all('table')[0]

    tickers = []
    names = []
    gics = []

    # Looping through the soup lxml text table format
    # and splitting each row as a individual string
    # and parsing string to retrieve the date,
    # open, and close information.
    i = 1
    for row in table.find_all('tr'):
        if i == 1:
            i += 1
            continue
        # Individual row stores current row item and delimits on '\n'
        individual_row = str(row).split('\n')
        # row_items is parsed string for each current row where each
        ticker = individual_row[1].split('">')[-1].split('<')[0]
        tickers.append(ticker)
        name = individual_row[2].split('">')[-1].split('<')[0]
        names.append(name)
        gic = individual_row[4].split('>')[1].split('<')[0]
        gics.append(gic)

    sp500 = pd.DataFrame({'Name': names, 'GIC': gics}, index = tickers)
    sp500.index.name = 'Tickers'
    return sp500

nasdaq = pd.read_csv('http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=NASDAQ&render=download', index_col = 0)[['Name','LastSale','IPOyear','Sector']]
nyse = pd.read_csv('http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=NYSE&render=download', index_col = 0)[['Name','LastSale','IPOyear','Sector']]
us_stocks = pd.concat([nyse,nasdaq], axis = 0).drop_duplicates()

In [239]:
'''
Function for pulling implied volatility from option slam for single ticker
'''

def optionslam_scrape(ticker):
    site = 'https://www.optionslam.com/earnings/stocks/' + ticker
    res = requests.get(site)
    soup = bs(requests.get(site).text, "lxml")
    soup = soup.prettify()
    earnings_dict = {'Ticker': ticker}
    
    # Check if there's weekly options
    curr7_implied = "Current 7 Day Implied Movement:"
    implied_move_weekly = "Implied Move Weekly:"
    nextearnings = "Next Earnings Date:"
    if curr7_implied not in soup:
        return 'No Weeklies'
    
    # Parsing if weekly options exist
    # Next earnings date and before or after
    earnings_start_string = "Next Earnings Date:"
    earnings_end_string = '</font>'
    raw_earnings_string = (soup.split(earnings_start_string))[1].split(earnings_end_string)[0].replace('\n','').strip()
    
    try:
        earnings_date = str((raw_earnings_string.split('<b>'))[1].split('<font size="-1">')).split("'")[1].strip()
    except:
        return 'Error Parsing'
    
    earnings_time = str(raw_earnings_string[-2:].strip()).strip()
    
    earnings_dict['Date'] = earnings_date
    earnings_dict['Earnings Time'] = earnings_time
    
    # Parsing 7 day implied move if weekly option exists
    ending_string = '<font size="-2">'
    curr_7 = (soup.split(curr7_implied))[1].split(ending_string)[0].replace('\n','').strip("").split("<td>")[-1].strip()
    earnings_dict['Current 7 Day Implied'] = curr_7
    
    # Parsing Weekly Implied move if weekly option exists
    if implied_move_weekly in soup:
        weekly_implied = (soup.split(implied_move_weekly))[1].split(ending_string)[0].replace('\n','').strip("").split("<td>")[-1].strip()
    else:
        weekly_implied = ''
    earnings_dict["Implied Move Weekly"] = weekly_implied
    
    return earnings_dict

# Function for calculating standard dev and price moves in terms of standard dev
# DF[[Adj Close]] Rolling Period --> DF[['Daily Vol','Daily Price Vol','Price Dev','Annual Vol']]
def price_devs(ticker, lookbackwindow, rollingperiod):
    # Define which online source one should use
    data_source = 'yahoo'
    
    end = dt.datetime.today()
    start_date = end - dt.timedelta(days = lookbackwindow)
    
    # User pandas_reader.data.DataReader to load the desired data. As simple as that.
    df = datareader.DataReader([ticker], data_source, start_date, end).sort_index()

    # Getting just the adjusted closing prices. This will return a Pandas DataFrame
    # The index in this DataFrame is the major index of the panel_data.
    df = df.ix['Close'].sort_index()
    
    df.columns = ['prices']
    df['prices_delta'] = df.prices - df.prices.shift(1)
    df['log_returns'] = np.log(df.prices) - np.log(df.prices.shift(1))
    df['daily_vol'] = st.rolling_std(df.log_returns, rollingperiod, ddof = 1)
    df['daily_vol_dollar'] = df.daily_vol*df.prices
    df['price_dev'] = df.prices_delta/df.daily_vol_dollar.shift(1)
    df['annual_vol'] = df.daily_vol*np.sqrt(252)
    return df


'''
Function for pulling options data from yahoo Input is a string, either 'call' or 'put' to
determine the contract type to pull from yahoo finance. The output is a dataframe of the latest
data from yahoo finance tagged with the current date-time. Output columns are pull date-time,
contract name, strike, last price, bid, ask volume, open interest, and IV (in decimal form).
'''
def yahoo_options(date, contract = 'put', ticker = 'SPX'):
    # date = dt.datetime(2018, 6, 8, 0, 0)
    raw_date = date - dt.timedelta(minutes = 7*60)
    unix_date = round(time.mktime(raw_date.timetuple()))
    # Using request to ping yahoo and retrieve the raw html
    # tables for calls and puts data for gspc
    if ticker == 'SPX':
        site = 'https://finance.yahoo.com/quote/%5EGSPC/options?p=%5EGSPC&date={0}'.format(unix_date)
    else:
        site = 'https://finance.yahoo.com/quote/{0}/options?p={0}&date={1}'.format(ticker, unix_date)
    res = requests.get(site)
    soup = bs(res.text, 'lxml')
    calls = soup.find_all('table')[0]
    puts = soup.find_all('table')[1]
    if contract == 'call':
        table = calls
    else:
        table = puts
        
    # parsing calls table
    
    # initiating data lists for storing column data
    dates = []
    names = []
    strikes = []
    lprices = []
    bids = []
    asks = []
    volumes = []
    open_interests = []
    iv = []
    current_time = dt.datetime.now()
    
    # starting with counter i so that the first row in the
    # specified table is noted as the header row
    i = 1
    for row in table.find_all('tr'):
        rowstring = str(row).split('\n')
        # if first row, store data as header labels for dataframe use
        if i == 1:
            header = [data.split('>')[1] for data in \
                        [rawstring.split('</span>')[0] for \
                         rawstring in rowstring[0].split('<span')[1:]]]
        else:
        # parsing other raw row strings to retrieve necessary data and append them
        # to their corresponding list.
            row_data = rowstring[0].split('data-reactid=')[3:]
            dates.append(current_time)
            names.append(row_data[0].split('title="')[1].split('"')[0])
            strikes.append(float(row_data[3].split('</a>')[0].split('>')[1].replace(',','')))
            lprices.append(float(row_data[4].split('</td>')[0].split('>')[1].replace(',','')))
            bids.append(float(row_data[5].split('</td>')[0].split('>')[1].replace(',','')))
            asks.append(float(row_data[6].split('</td>')[0].split('>')[1].replace(',','')))
            volumes.append(float(row_data[-3].split('</td>')[0].split('>')[1].replace(',','')))
            open_interests.append(float(row_data[-2].split('</td>')[0].split('>')[1].replace(',','')))
            iv.append(float(row_data[-1].split('</td>')[0].split('>')[1].replace(',','').replace('%',''))/100)
        i += 1

    return pd.DataFrame({'Pull Date': dates,
                          contract + ' ' + header[0]: names,
                          header[2]: strikes,
                          header[3]: lprices,
                          header[4]: bids,
                          header[5]: asks,
                          header[-3]: volumes,
                          header[-2]: open_interests,
                          header[-1]: iv})

def fundamentals(ticker):
    
    site = 'https://finance.yahoo.com/quote/{0}?p={0}'.format(ticker)

    res = requests.get(site)
    soup = bs(res.text, 'lxml')
    table = soup.find_all('table')[1]
    sum_dict = {}

    # Looping through the soup lxml text table format
    # and splitting each row as a individual string
    # and parsing string to retrieve the date,
    # open, and close information.


    for row in table.find_all('tr'):
        # Individual row stores current row item and delimits on '\n'
        individual_row = str(row).split('\n')[0]

        # row_items is parsed string for each current row where each
        # item in list is the date, open, high, low, close, and volume
        row_items = individual_row.split('<span data-reactid=')[1].split('"><!-- react-text: ')
        if len(row_items) > 1:
            sum_item = row_items[0].split('>')[1].split('<')[0]
            sum_value = row_items[1].split('-->')[1].split('<')[0]
        elif 'YIELD' in row_items[0]:
            try:
                temp_val = row_items[0].split('-value">')[1].split("</td>")[0]
                div_amount = float(temp_val.split(' ')[0])
                div_yield = float(temp_val.split(' ')[1].replace('(','').replace(')','').replace('%',''))

                sum_dict['Div'] = div_amount
                sum_dict['Yield'] = div_yield
            except:
                sum_dict['Div'] = np.nan
                sum_dict['Yield'] = np.nan

        sum_dict[sum_item] = sum_value

    return pd.DataFrame(sum_dict, index = [ticker])

# Function to return fundametal data of a ticker list
def get_fundas(ticker_lst):
    fund_lst = []
    for tick in ticker_lst:
        fund_lst.append(fundamentals(tick))
    return pd.concat(fund_lst,axis = 0)

# Function to pull straddled view of options of a ticker
def get_option_chain(ticker, date):
    putframe = yahoo_options(date, contract = 'put', ticker = ticker)
    callframe = yahoo_options(date, contract = 'call', ticker = ticker)
    calls = callframe[['Ask','Bid','Implied Volatility','Last Price','Open Interest','Strike','Volume']]
    puts = putframe[['Ask','Bid','Implied Volatility','Last Price','Open Interest','Strike','Volume']]
    return puts.merge(calls, how = 'inner', on = 'Strike', suffixes=('_C', '_P'))

# Function historical data from alpha advantage
def historical_data(ticker, window = 252, outsize = 'full'):
    alphavantage_link = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={0}&apikey=5HZEUI5AFJB06BUK&datatype=csv&outputsize={1}'.format(ticker, outsize)
    stockframe = pd.read_csv(alphavantage_link, index_col = 0).sort_index()[['open', 'close']]
    stockframe['daily_ret'] = np.log(stockframe['close']/stockframe['close'].shift(1))
    stockframe['intra_ret'] = np.log(stockframe['close']/stockframe['open'])
    stockframe['ovrnt_ret'] = np.log(stockframe['open']/stockframe['close'].shift(1))
    stockframe['daily_vol'] = stockframe.daily_ret.rolling(window=20,center=False).std()
    stockframe['intra_vol'] = stockframe.intra_ret.rolling(window=20,center=False).std()
    stockframe['ovrnt_vol'] = stockframe.ovrnt_ret.rolling(window=20,center=False).std()
    stockframe['daily_ann'] = stockframe.daily_vol*np.sqrt(252)
    stockframe['intra_ann'] = stockframe.intra_vol*np.sqrt((24/6.5)*252)
    stockframe['ovrnt_ann'] = stockframe.ovrnt_vol*np.sqrt((24/17.5)*252)
    stockframe['oc_diff'] = stockframe.close - stockframe.open
    stockframe['daily_dollar_vol'] = stockframe.daily_vol*stockframe.close.shift(1)
    stockframe['daily_dollar_std'] = np.abs(stockframe.oc_diff/stockframe.daily_dollar_vol)

    return stockframe.tail(window)

# Function for building a dataframe of volatilities
# Daily, Intraday, Overnight
def current_volatility(ticker_list):
    
    rows = []
    for tick in ticker_list:
        try:
            curr_vol = historical_data(tick, window = 1, outsize = 'compact')[['daily_ann','intra_ann','ovrnt_ann']]
            curr_vol.index.name = 'Tickers'
            curr_vol.index = [tick]
            rows.append(curr_vol)
        except:
            continue
        
    return pd.concat(rows, axis = 0)

# Function for pulling S&P500 data and calculating volatilities
def sp500_filter():
    sp500 = pull_sp500_list()
    sp500_vols = current_volatility(sp500.index.tolist())
    df = pd.concat([sp500_vols, sp500], axis = 1).dropna()
    
    return df

In [258]:
us_vol = current_volatility(us_stocks.index.tolist())

In [ ]:
sp500_len = len(sp500)
end_range = 100
start_range = 0
i = 0
batch_list = []
while sp500_len > 100:
    tick_lst = sp500[start_range:end_range].index.tolist()
    tickers = str(tick_lst).replace('[','').replace(']','').replace("'","").replace(" ","")
    alphavantage_link = 'https://www.alphavantage.co/query?function=BATCH_STOCK_QUOTES&symbols={0}&apikey=5HZEUI5AFJB06BUK&datatype=csv'.format(tickers)
    batch_list.append(pd.read_csv(alphavantage_link, index_col = 0)[['price']])
    start_range = end_range
    end_range = end_range + 100
    sp500_len = sp500_len - 100
    i += 1
start_range = i*100
tick_lst = sp500[start_range:(start_range + sp500_len)].index.tolist()
tickers = str(tick_lst).replace('[','').replace(']','').replace("'","").replace(" ","")
alphavantage_link = 'https://www.alphavantage.co/query?function=BATCH_STOCK_QUOTES&symbols={0}&apikey=5HZEUI5AFJB06BUK&datatype=csv'.format(tickers)
batch_list.append(pd.read_csv(alphavantage_link, index_col = 0)[['price']])
batch_quotes = pd.concat(batch_list, axis = 0)
sp500_df = pd.concat([sp500, batch_quotes], axis = 1)
sp500_df['Intra_Daily_Ratio'] = sp500_df['intra_ann']/sp500_df['daily_ann']
sp500_df = sp500_df.sort_values(['Intra_Daily_Ratio'], ascending = False)

In [308]:
current_volatility(['FB'])

daily_ann  intra_ann  ovrnt_ann
FB   0.376679   0.405024   0.346539

In [269]:
0.614183/np.sqrt(126)*300

16.414731134149854

In [309]:
tsla = historical_data('FB')
tsla

open   close  daily_ret  intra_ret  ovrnt_ret  daily_vol  \
timestamp                                                                  
2017-05-09  151.4900  150.48  -0.003847  -0.006689   0.002843   0.007936   
2017-05-10  150.2300  150.29  -0.001263   0.000399  -0.001663   0.007570   
2017-05-11  150.3100  150.04  -0.001665  -0.001798   0.000133   0.007540   
2017-05-12  150.4000  150.33   0.001931  -0.000466   0.002396   0.007461   
2017-05-15  150.1700  150.19  -0.000932   0.000133  -0.001065   0.007086   
2017-05-16  150.1100  149.78  -0.002734  -0.002201  -0.000533   0.007062   
2017-05-17  148.0000  144.85  -0.033469  -0.021514  -0.011955   0.010638   
2017-05-18  144.7200  147.66   0.019214   0.020111  -0.000898   0.011206   
2017-05-19  148.4500  148.06   0.002705  -0.002631   0.005336   0.011198   
2017-05-22  148.0800  148.24   0.001215   0.001080   0.000135   0.010902   
2017-05-23  148.5200  148.07  -0.001147  -0.003034   0.001887   0.010816   
2017-05-24  148.5100  150.04   0.013217   0.010250   0.002967   0.011181   
2017-05-25  150.3000  151.96   0.012715   0.010984   0.001731   0.011388   
2017-05-26  152.2300  152.13   0.001118  -0.000657   0.001775   0.010773   
2017-05-30  151.9700  152.38   0.001642   0.002694  -0.001052   0.010266   
2017-05-31  152.7000  151.46  -0.006056  -0.008154   0.002098   0.010338   
2017-06-01  151.7500  151.53   0.000462  -0.001451   0.001913   0.010242   
2017-06-02  151.8500  153.61   0.013633   0.011524   0.002110   0.010571   
2017-06-05  153.6400  153.63   0.000130  -0.000065   0.000195   0.010509   
2017-06-06  153.4100  152.81  -0.005352  -0.003919  -0.001433   0.010553   
2017-06-07  153.2700  153.12   0.002027  -0.000979   0.003006   0.010505   
2017-06-08  154.0800  154.71   0.010330   0.004080   0.006250   0.010699   
2017-06-09  154.7700  149.60  -0.033587  -0.033975   0.000388   0.013262   
2017-06-12  148.1700  148.44  -0.007784   0.001821  -0.009605   0.013359   
2017-06-13  150.1500  150.68   0.014978   0.003524   0.011454   0.013807   
2017-06-14  151.2600  150.25  -0.002858  -0.006700   0.003842   0.013808   
2017-06-15  147.6700  149.80  -0.003000   0.014321  -0.017321   0.011368   
2017-06-16  149.5900  150.64   0.005592   0.006995  -0.001403   0.010648   
2017-06-19  151.7100  152.87   0.014695   0.007617   0.007078   0.011078   
2017-06-20  152.8800  152.25  -0.004064  -0.004129   0.000065   0.011150   
...              ...     ...        ...        ...        ...        ...   
2018-03-27  156.3100  152.22  -0.050222  -0.026514  -0.023708   0.023732   
2018-03-28  151.6500  153.03   0.005307   0.009059  -0.003752   0.023840   
2018-03-29  155.1500  159.79   0.043226   0.029468   0.013758   0.026351   
2018-04-02  157.8100  155.39  -0.027922  -0.015454  -0.012469   0.026756   
2018-04-03  156.5500  156.11   0.004623  -0.002815   0.007437   0.026106   
2018-04-04  152.0250  155.10  -0.006491   0.020025  -0.026516   0.026091   
2018-04-05  161.5600  159.34   0.026970  -0.013836   0.040807   0.026429   
2018-04-06  157.7300  157.20  -0.013521  -0.003366  -0.010156   0.026468   
2018-04-09  157.8200  157.93   0.004633   0.000697   0.003936   0.026071   
2018-04-10  157.9300  165.04   0.044036   0.044036   0.000000   0.028545   
2018-04-11  165.3600  166.32   0.007726   0.005789   0.001937   0.028591   
2018-04-12  166.9800  163.87  -0.014840  -0.018801   0.003960   0.028385   
2018-04-13  164.5800  164.52   0.003959  -0.000365   0.004323   0.028457   
2018-04-16  165.7249  164.83   0.001882  -0.005415   0.007297   0.028369   
2018-04-17  165.8300  168.66   0.022970   0.016922   0.006049   0.024657   
2018-04-18  166.8800  166.36  -0.013731  -0.003121  -0.010610   0.024159   
2018-04-19  166.2000  168.10   0.010405   0.011367  -0.000962   0.024222   
2018-04-20  167.7900  166.28  -0.010886  -0.009040  -0.001846   0.023553   
2018-04-23  167.2700  165.84  -0.002650  -0.008586   0.005936   0.022149   
2018-04-24  165.4300  159.69  -0.037789  -0.035314  -0.002475   

In [310]:
len(tsla[tsla.daily_dollar_std >= 1])

57